# Titanic Dataset - 数据预处理

这个notebook主要进行以下工作：
1. 数据加载和初步探索
2. 缺失值处理
3. 特征工程
4. 数据清洗和转换

In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置显示选项
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
%matplotlib inline

C:\Users\Administrator\AppData\Local\Temp\ipykernel_29700\760322174.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
# 加载数据
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

# 显示训练集的基本信息
print("训练集形状:", train_data.shape)
train_data.head()

训练集形状: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 查看数据基本信息
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# 检查缺失值
print("训练集缺失值情况:")
print(train_data.isnull().sum())

print("\n测试集缺失值情况:")
print(test_data.isnull().sum())

训练集缺失值情况:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

测试集缺失值情况:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [5]:
# 数值特征的统计描述
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 特征工程

接下来我们将：
1. 处理缺失值
2. 提取特征
3. 转换分类变量

In [6]:
# 合并训练集和测试集进行特征工程
def preprocess_data(df):
    # 复制数据框
    data = df.copy()
    # 填充缺失值
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)
    # 提取称谓
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    # 简化称谓
    title_mapping = {'Mr': 'Mr','Miss': 'Miss','Mrs': 'Mrs','Master': 'Master','Dr': 'Other','Rev': 'Other','Col': 'Other','Major': 'Other','Mlle': 'Miss','Countess': 'Other','Ms': 'Miss','Lady': 'Other','Jonkheer': 'Other','Don': 'Other','Dona': 'Other','Mme': 'Mrs','Capt': 'Other','Sir': 'Other'
    }
    data['Title'] = data['Title'].map(title_mapping)
    # 创建家庭规模特征
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    # 创建是否独自一人特征
    data['IsAlone'] = 0
    data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1
    # 对分类变量进行编码
    data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'Title'])
    # 删除不需要的列
    data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1, inplace=True)
    return data
# 处理训练集
processed_train = preprocess_data(train_data)
processed_test = preprocess_data(test_data)

print("处理后的训练集形状:", processed_train.shape)
processed_train.head()

处理后的训练集形状: (891, 18)


,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other
0,0,3,22.0,1,0,7.2500,2,0,False,True,False,False,True,False,False,True,False,False
1,1,1,38.0,1,0,71.2833,2,0,True,False,True,False,False,False,False,False,True,False
2,1,3,26.0,0,0,7.9250,1,1,True,False,False,False,True,False,True,False,False,False
3,1,1,35.0,1,0,53.1000,2,0,True,False,False,False,True,False,False,False,True,False
4,0,3,35.0,0,0,8.0500,1,1,False,True,False,False,True,False,False,True,False,False


In [7]:
# 保存处理后的数据
processed_train.to_csv('data/processed_train.csv', index=False)
processed_test.to_csv('data/processed_test.csv', index=False)

print("数据预处理完成，已保存处理后的数据文件。")

数据预处理完成，已保存处理后的数据文件。
